일반화 성능도 있기 때문에 전체 데이터셋에 대해서 성능 평가를 해봐야 함

In [1]:
import pickle
with open('../models/BernoulliNB norm l30_r15.pkl', 'rb') as f:
    classifier = pickle.load(f)
    
from py.utils import load_data
head = 'l30_r15'
directory = '../data/'
x, y, x_words, vocabs = load_data(head, directory)

x shape = (15106, 2770)
y shape = (15106,)
# features = 2770
# L words = 15106


In [2]:
word_frequency = x.sum(axis=1)
word_frequency

matrix([[  88],
        [  61],
        [2198],
        ..., 
        [ 224],
        [  57],
        [ 448]])

In [3]:
def get_sample_l(ri, topk=5):
    nonzero = x[:,ri].nonzero()[0]
    base = min(50, len(nonzero)//2)
    return [x_words[l] for l in sorted(nonzero, key=lambda x:word_frequency[x,0], reverse=True)[base:base+topk]]

coefficient = log prob

==> exp(coefficient) = prob

In [4]:
import numpy as np
np.exp(classifier.class_log_prior_)

array([ 0.16589435,  0.83410565])

In [5]:
import numpy as np

base = 0
topk = 100
for r, coef in sorted(enumerate(classifier.coef_[0]), key=lambda x:x[1], reverse=True)[base :base + topk]:
    print('%s - %s (logp= %.3f, p= %.3f)' % (get_sample_l(r), vocabs[r], coef, np.exp(coef)))

['몸', '무엇', '얘기', '아버지', '모습'] - 의 (logp= -0.192, p= 0.826)
['여자', '몸', '무엇', '얘기', '아버지'] - 에 (logp= -0.266, p= 0.767)
['사회', '여기', '지금', '이야기', '안'] - 도 (logp= -0.560, p= 0.571)
['죽', '다음', '이상', '아이들', '삶'] - 을 (logp= -0.593, p= 0.552)
['책', '이름', '이들', '세상', '한국'] - 이 (logp= -0.604, p= 0.546)
['모습', '길', '죽', '다음', '이상'] - 은 (logp= -0.686, p= 0.504)
['밖', '쪽', '물', '적', '글'] - 과 (logp= -0.803, p= 0.448)
['얘기', '아버지', '모습', '길', '자리'] - 에서 (logp= -0.860, p= 0.423)
['명', '끝', '주장', '뜻', '입'] - 으로 (logp= -1.010, p= 0.364)
['예', '역사', '시', '존재', '머리'] - 를 (logp= -1.012, p= 0.364)
['나가', '누구', '문화', '예', '역사'] - 가 (logp= -1.031, p= 0.357)
['글', '개', '남자', '학교', '생활'] - 로 (logp= -1.077, p= 0.341)
['자리', '삶', '뒤', '나라', '정부'] - 는 (logp= -1.087, p= 0.337)
['예', '역사', '시', '존재', '머리'] - 와 (logp= -1.180, p= 0.307)
['아버지', '모습', '길', '자리', '다음'] - 인 (logp= -1.227, p= 0.293)
['내용', '책', '이름', '이들', '세상'] - 이나 (logp= -1.295, p= 0.274)
['자기', '사랑', '데', '미국', '관계'] - 하고 (logp= -1.419, p= 0.242)


## BernoulliNB classifier code

    def _joint_log_likelihood(self, X):
        """Calculate the posterior log probability of the samples X"""
        check_is_fitted(self, "classes_")

        X = check_array(X, accept_sparse='csr')

        if self.binarize is not None:
            X = binarize(X, threshold=self.binarize)

        n_classes, n_features = self.feature_log_prob_.shape
        n_samples, n_features_X = X.shape

        if n_features_X != n_features:
            raise ValueError("Expected input with %d features, got %d instead"
                             % (n_features, n_features_X))

        neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
        # Compute  neg_prob · (1 - X).T  as  ∑neg_prob - X · neg_prob
        jll = safe_sparse_dot(X, (self.feature_log_prob_ - neg_prob).T)
        jll += self.class_log_prior_ + neg_prob.sum(axis=1)

        return jll

### Bernoulli Naive Bayes 역시 infrequent feature가 low scale

classification rule: $ P(x_i \mid y) = P(i \mid y) x_i + (1 - P(i \mid y)) (1 - x_i) $

class y에서 feature i의 생성확률을 만들기 때문에 infrequent 한 feature가 작은 coefficient를 지님

In [6]:
np.exp(classifier.coef_[0,vocabs.index('해줌으로써')])

0.0011109347722583723